In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

/Users/mattzhang/py2_kernel/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


# Loading and Cleaning Data

We want to predict, based on a listing's properties and price, and the economics of the surrounding area, whether that listing is likely to be booked or not.

In [2]:
listings = pd.read_csv("Data/listings.csv")

In [3]:
pd.set_option('display.max_columns', None)
listings

,accommodates,amenities,availability_30,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,has_availability,host_id,id,instant_bookable,latitude,longitude,metropolitan,name,price,property_type,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,room_type,state,weekly_price,zipcode
0,2.0,"{""Cable TV"",""Wireless Internet"",""Air condition...",24,1.0,Real Bed,1.0,1.0,moderate,sunnysidebronx,NaN,119445,7949480,f,40.852054,-73.788680,NYC,City Island Sanctuary relaxing BR & Bath w Par...,$99.00,House,10.0,10.0,10.0,10.0,100.0,10.0,Private room,NY,NaN,10464
1,4.0,"{TV,Internet,""Wireless Internet"",""Air conditio...",30,1.0,Real Bed,1.0,1.0,flexible,sunnysidebronx,NaN,9117975,16042478,t,40.853491,-73.788607,NYC,WATERFRONT STUDIO APARTMENT,$200.00,Apartment,NaN,NaN,NaN,NaN,NaN,NaN,Private room,NY,NaN,10464
2,4.0,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",30,3.0,Real Bed,3.0,3.0,strict,sunnysidebronx,NaN,9815788,1886820,f,40.841144,-73.783052,NYC,Quaint City Island Community.,$300.00,House,NaN,NaN,NaN,NaN,NaN,NaN,Entire home/apt,NY,NaN,10464
3,3.0,"{TV,Internet,""Wireless Internet"",""Air conditio...",8,1.0,Real Bed,1.0,1.0,strict,long island city,NaN,13886510,6627449,f,40.849775,-73.786609,NYC,Large 1 BDRM in Great location,$125.00,Apartment,10.0,10.0,10.0,10.0,93.0,10.0,Entire home/apt,NY,$775.00,10464
4,4.0,"{Internet,""Wireless Internet"",""Air conditionin...",17,1.0,Real Bed,1.0,1.0,moderate,sunnysidebronx,NaN,28811542,5557381,t,40.850024,-73.789328,NYC,Quaint City Island Home,$69.00,House,10.0,10.0,10.0,10.0,97.0,10.0,Private room,NY,$350.00,10464
5,2.0,"{TV,""Wireless Internet"",""Air conditioning"",Kit...",23,1.0,Real Bed,0.0,1.0,moderate,sunnysidebronx,NaN,403032,9147025,f,40.844870,-73.789541,NYC,Cozy City Island Cottage,$125.00,House,10.0,10.0,10.0,10.0,97.0,10.0,Entire home/apt,NY,$550.00,10464
6,4.0,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",15,1.0,Real Bed,1.0,2.0,flexible,long island city,NaN,56714504,11675715,t,40.851391,-73.784139,NYC,Cozy 1 BR Basement Apartment,$85.00,House,10.0,10.0,10.0,10.0,98.0,10.0,Entire home/apt,NY,NaN,10464
7,3.0,"{""Cable TV"",Internet,""Wireless Internet"",""Air ...",5,1.0,Real Bed,1.0,2.0,strict,sunnysidebronx,NaN,3684360,715270,f,40.859559,-73.870669,NYC,2 Beds/Queen & Full Beautiful Room 40 minsT.Sq...,$39.00,Apartment,9.0,9.0,9.0,9.0,90.0,9.0,Private room,NY,NaN,10467
8,5.0,"{TV,Internet,""Wireless Internet"",""Air conditio...",17,1.0,Real Bed,1.0,1.0,moderate,sunnysidebronx,NaN,11305944,17876530,f,40.868682,-73.854828,NYC,Spacious Garden Apartment,$95.00,House,10.0,10.0,10.0,10.0,100.0,10.0,Entire home/apt,NY,NaN,10469
9,8.0,"{TV,Internet,""Wireless Internet"",""Air conditio...",12,1.0,Real Bed,1.0,3.0,strict,sunnysidebronx,NaN,873273,182177,t,40.864658,-73.857087,NYC,PRIVATE FLAT / APARTMENT- $SPECIAL$,$125.00,Apartment,10.0,9.0,10.0,9.0,92.0,9.0,Entire home/apt,NY,NaN,10469


In [4]:
listings = listings[['accommodates', 'bathrooms', 'beds', 'availability_30']]

# Neural Net

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [6]:
# set random seed for reproducibility
random_seed = 13
np.random.seed(random_seed)

In [7]:
# shuffle and split into test and train
from sklearn.utils import shuffle
listings = shuffle(listings)

from __future__ import division
X = listings.drop('availability_30', 1)
y = listings['availability_30']/30

testRatio = 0.2
nSamples = len(X)
nTest = int(nSamples * testRatio)

X_test = X[:nTest].values
y_test = y[:nTest].values
X_train = X[nTest:].values
y_train = y[nTest:].values
del listings, X, y

In [8]:
model = Sequential()
model.add(Dense(12, input_dim=X_train.shape[1], init='normal', activation='relu'))
model.add(Dense(8, init='normal', activation='relu'))
model.add(Dense(1, init='normal'))
model.compile(loss='mean_squared_error', optimizer='adam')

/Users/mattzhang/py2_kernel/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, activation="relu", kernel_initializer="normal", input_dim=3)`
  from ipykernel import kernelapp as app
/Users/mattzhang/py2_kernel/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_initializer="normal")`
  app.launch_new_instance()
/Users/mattzhang/py2_kernel/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal")`


In [14]:
y_test

array([ 0.13333333,  1.        ,  0.13333333, ...,  0.        ,
        0.03333333,  0.96666667])

In [9]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=100)

Train on 47860 samples, validate on 11964 samples
Epoch 1/5
47860/47860 [==============================] - 1s - loss: nan - val_loss: nan
Epoch 2/5
47860/47860 [==============================] - 1s - loss: nan - val_loss: nan
Epoch 3/5
47860/47860 [==============================] - 1s - loss: nan - val_loss: nan
Epoch 4/5
47860/47860 [==============================] - 1s - loss: nan - val_loss: nan
Epoch 5/5
47860/47860 [==============================] - 1s - loss: nan - val_loss: nan


In [10]:
# create and compile model
def model():
    model = Sequential()
    model.add(Dense(12, input_dim=X_train.shape[1], init='normal', activation='relu'))
    model.add(Dense(8, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [11]:
# fit the model
estimator = KerasRegressor(build_fn=model, nb_epoch=10, batch_size=5)
kfold = KFold(n_splits=10, random_state=random_seed)
results = cross_val_score(estimator, X_train.values, y_train.values, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

AttributeError: 'numpy.ndarray' object has no attribute 'values'

# BDT